In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# plot feature importance using built-in function
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_importance



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. if you liked my work, please upvote this kernel since it will keep me motivated. On the other hand please share your pozitif or negatif comments and ideas about kernel with me.
# 2.Our Goals: Increase scores and some different metrics with resampling  and gridsearch methods.
    


# Content
# 1. [Data Visualization](#visualizations)

# 2. [Build Model](#model)
*      [Model conclusion ](#confusion)

# 3. [Feature Engineering](#feature_engineering)

*      [Feature Extraction ](#feature_extraction)
*      [Create Different datasets ](#different_data)
*      [Dimension reduction the Datasets ](#Dim_reduction)

# 4. [Under Sampling Process](#under_samp)
*      [Enn--- Tomeklinks--- Nearmiss ](#techniques)
*      [GridSearch](#GridSearch)
*      [F1 Scores After Under Sampling ](#F1_scores)


# 5. [Over Sampling](#over_sampling)
*      [F1 Scores After Over Sampling ](#over_sampling_f1)

# 6. [Roc Curve and models conclusion](#roc_curve)

*      [Conclusions ](#Conclusions)


1. ID: ID of each client
1. LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
1. SEX: Gender (1=male, 2=female)
1. EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
1. MARRIAGE: Marital status (1=married, 2=single, 3=others)
1. AGE: Age in years
1. PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, 8=payment delay for eight months, 9=payment delay for nine months and above)
1. PAY_2: Repayment status in August, 2005 (scale same as above)
1. PAY_3: Repayment status in July, 2005 (scale same as above)
1. PAY_4: Repayment status in June, 2005 (scale same as above)
1. PAY_5: Repayment status in May, 2005 (scale same as above)
1. PAY_6: Repayment status in April, 2005 (scale same as above)
1. BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
1. BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
1. BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
1. BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
1. BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
1. BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
1. PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
1. PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
1. PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
1. PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
1. PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
1. PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
1. default.payment.next.month: Default payment (1=yes, 0=no)

In [ ]:
data = pd.read_csv("../input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv")

In [ ]:
data.columns

In [ ]:
data = data.rename(columns={"default.payment.next.month": "target"})

In [ ]:
data.shape

In [ ]:
data.info()

Now, we know that there is no any missing value. Great!

In [ ]:
data.head(10)

id number is not necessary for us...

In [ ]:
data = data.drop(columns=["ID"], axis=1)

<a id="visualizations"></a>
# Data Visualizations

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x="target", data=data, palette="bwr")
plt.xticks(np.arange(2),('no', "yes"))
plt.xlabel("Default Payent" , color="red", alpha=0.7, size=22)
plt.show()

print(data["target"].value_counts())
print((100*6636)/30000,"percantage of customers will default")

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x="SEX", data=data, palette="bwr")
plt.xticks(np.arange(2), ('Male', 'Female'))
plt.xlabel("Sex" , color="red", alpha=0.7, size=22)
plt.show()
numberofsex=data["SEX"].value_counts()

print("Male count: {} \nFamele count: {}".format(numberofsex[1], numberofsex[2]))

In [ ]:
groupby1 = data.groupby("SEX")["LIMIT_BAL"].mean()
print(groupby1)

plt.figure(figsize=(12,6))
plt.style.use("seaborn")
groupby1.plot.bar(color="#10CCBC")
plt.xticks(np.arange(2), ('Male', 'Female'), color="#CC1057")
plt.xlabel("Sex" , color="red", alpha=0.7, size=22)
plt.show()

In [ ]:
print("Age values are between {} and {}.".format(data["AGE"].min(), data["AGE"].max()))
limit_mean=data.groupby('AGE')['LIMIT_BAL'].mean()
print(limit_mean)
limit_mean=limit_mean.values.reshape(-1,1)

In [ ]:
plt.figure(figsize=(16,8))
plt.style.use("ggplot")
plt.grid(True)
plt.plot(np.arange(21,77,1),limit_mean, color="purple", alpha=0.7)
plt.title("Average credit limit and Age", color="red", size=22)
plt.xlabel("Age", color="black", alpha=0.8, size=16)
plt.xlim(20,80)
plt.show()


Lets we look at any relationship between age and repayment.

In [ ]:
groupby_2=data.groupby('AGE')['target'].mean()

plt.figure(figsize=(16,8))
plt.style.use("ggplot")
plt.grid(True)
plt.plot(np.arange(21,77,1),groupby_2, color="black", alpha=0.7)
plt.hlines(0.25,xmin=20, xmax=80, color="red", alpha=0.4, linewidth=3)
plt.title("repayment_value and Age", color="red", size=22)
plt.xlabel("Age", color="black", alpha=0.8, size=16)
plt.annotate('Critical line',xy=(120, 200), xycoords='figure points', xytext=(80, 160),textcoords='offset points',
            arrowprops=dict(facecolor='red', shrink=0.05, alpha=0.6))
plt.xlim(20,80)
plt.show()

In [ ]:
print(data["EDUCATION"].value_counts(dropna=False, ascending=False))

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x="EDUCATION", data=data, palette="dark")
plt.xticks(np.arange(7), ("graduate school" if i==1 else "university" if i==2 else "high school" if i==3 else 
                          "other1" if i==4 else "other2" if i==5 else "other3" if i==6 else "other" for i in range(7)), color="black")
plt.xlabel("Education", color="red", size=22, alpha=0.7)
plt.show()

In [ ]:
print(data["target"].value_counts())
#(Yes = 1, No = 0) 

In [ ]:
groupby_3=data.groupby('EDUCATION')['target'].mean()
groupby_3=pd.DataFrame(groupby_3)
groupby_3

# Swarmplot takes more time... lets use that only 4000 sample for this visualization

In [ ]:
data2=data.iloc[:4000,:] #because its take a lot of time.
sns.set(style="dark")
plt.figure(figsize=(14,10))
sns.swarmplot(x ="EDUCATION", y = "LIMIT_BAL", hue="SEX",  
              data = data2, color="green")
plt.ylabel("Education levels", color="red")
plt.ylabel("Credit Limits ", color="red")
plt.legend()
plt.show() 

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x="SEX", data=data, palette="dark")
plt.xticks(np.arange(2), ('Married', 'single'))
plt.xlabel("Marriage or Not" , color="red", alpha=0.7, size=22)
plt.show()
marriage=data["MARRIAGE"].value_counts()
print("Marriage count: {} \nSingle count: {}".format(marriage[1], marriage[2]))

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(data.corr(),annot=True,fmt='.2f',color='red',cmap='coolwarm')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = data.iloc[:,:23]
y = data.iloc[:,23]
print(X.columns)
y=pd.DataFrame(y)

clf = RandomForestClassifier()
clf.fit(X,y)

In [ ]:
importance = pd.DataFrame({'Feature': X.columns, 'Feature importance': clf.feature_importances_})
importance = importance.sort_values(by='Feature importance',ascending=False)
#print(importance)

plt.figure(figsize = (12,8))
plt.title('Features importance for RandomForestClassifier',fontsize=18, color="red")
plt.bar(x='Feature',height='Feature importance',data=importance, alpha = 0.8, edgecolor="black")
plt.xticks(rotation=80)
plt.show()

<a id="model"></a>

# Build models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

we controlled disturbitions and we can see the numbers are so close each other. Nice!

In [ ]:
X=data.drop(columns=['target'],axis=1)
y=data.iloc[:,23]


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=35)
print("X_train:" , len(X_train))
print("X_test:", len(X_test))
print("y_train", len(y_train))
print("y_test", len(y_test))


#y_train ve y_test deki 0 ve 1 değerleri.
print("percantage of target=0 :%",round(len(data[data["target"]==0])/len(data),3))
print("percantage of target=0 :%",round(len(y_train[y_train==0]) / len(y_train),3))
print("percantage of target=0 :%",round(len(y_test[y_test==0]) / len(y_test),3))

# Are you realize any mistakes about logistic regression ?

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
acc_log_train=log_reg.score(X_train,y_train)*100
acc_log_test=log_reg.score(X_test,y_test)*100
y_pred_log = log_reg.predict(X_test)

print('Training accuracy : % {}'.format(acc_log_train))
print('Test accuracy : % {}'.format(acc_log_test))

score_log=accuracy_score(y_pred_log,y_test)*100
print("Logistic Regression SCORE:",score_log)

print("Log_Reg confusion matrix")
cm_log = confusion_matrix(y_test, y_pred_log)
print(cm_log)

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
acc_rfc_train = rfc.score(X_train, y_train)*100
acc_rfc_test = rfc.score(X_test, y_test)*100
y_pred_rfc = rfc.predict(X_test)

print("Training accuracy : % {}".format(acc_rfc_train))
print("Test Accuracy: % {}".format(acc_rfc_test))

score_rfc=accuracy_score(y_pred_rfc,y_test)*100
print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc))

print("RandomForestClassifier Confusion matrix")
cm_rfc = confusion_matrix(y_test, y_pred_rfc)
print(cm_rfc)

In [ ]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)
acc_xgb_train = xgb.score(X_train, y_train)*100
acc_xgb_test = xgb.score(X_test, y_test)*100
y_pred_xgb =xgb.predict(X_test)

print("Training accuracy : % {}".format(acc_xgb_train))
print("Test Accuracy: % {}".format(acc_xgb_test))

score_xgb=accuracy_score(y_pred_xgb,y_test)*100
print("XgboostClassifier SCORE:",score_xgb)

print("Xgb Confusion matrix")
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print(cm_xgb)

In [ ]:
from sklearn.model_selection import cross_val_score

model_list=[log_reg, rfc, xgb]

all_cross_scores=[]


for i in model_list:
    scores=cross_val_score(i, X_train, y_train, cv=5, n_jobs=-1)
    print(str(i),", scores: ", scores)
    all_cross_scores.append(scores)  
    print("Mean cross validation score of {} models : {:.2f}".format(i, scores.mean() ))
    print("*"* 25)

In [ ]:
log_cross_score = all_cross_scores[0]
rfc_cross_score = all_cross_scores[1]
xgb_cross_score = all_cross_scores[2]

plt.style.use("ggplot")
fig, axs = plt.subplots(1, 3, figsize=(18,4))

axs[0].plot(log_cross_score, linewidth=3, color="black", marker='o')
axs[0].hlines(log_cross_score.mean(),xmin=0, xmax=4, color="red", alpha=0.4, linewidth=2)
axs[0].set_ylabel("score", color="red", size=24)
axs[0].set_title("Logistic Regression", color="#FF69B4")
axs[0].grid(True)

axs[1].plot(rfc_cross_score, linewidth=3, color="black", marker='o')
axs[1].hlines(rfc_cross_score.mean(),xmin=0, xmax=4, color="red", alpha=0.4, linewidth=2)
axs[1].set_xlabel("iterations",  color="red", size=24)
axs[1].set_title("RandomForest", color="#FF69B4")
axs[1].grid(True)

axs[2].plot(xgb_cross_score, linewidth=3, color="black", marker='o')
axs[2].set_title("XGBoost", color="#FF69B4")
axs[2].hlines(xgb_cross_score.mean(),xmin=0, xmax=4, color="red", alpha=0.4, linewidth=2)
axs[2].grid(True)

<a id="confusion"></a>

# Actually the accuracy score is not our aim. We find out customers who in default. as a conclusion we use confusion matrix to have a better perspective.

In [ ]:
from sklearn.metrics import plot_confusion_matrix

fig, axs = plt.subplots(1, 3, figsize=(24,10))

plot_confusion_matrix(log_reg, X_test, y_test,cmap=plt.cm.Blues, ax=axs[0],display_labels=("not default", "default"))
axs[0].set_title("Logistic Regression",fontsize=18, color="#CC1057")
axs[0].grid(False)


plot_confusion_matrix(rfc, X_test, y_test,cmap=plt.cm.PuBuGn, ax=axs[1],display_labels=("not default", "default"))
axs[1].set_title("RandomForestClassifier" ,fontsize=18, color="#10ABCC")
axs[1].grid(False)

plot_confusion_matrix(xgb, X_test, y_test,cmap=plt.cm.magma, ax=axs[2],display_labels=("not default", "default"))
axs[2].set_title("XGBoost Classifier",fontsize=18, color="#96CC10")
axs[2].grid(False)


In [ ]:
print(all_cross_scores)

In [ ]:
data_score={"names":["Log_reg","RandomFC","XGBoost"], "scores":[score_log,score_rfc,score_xgb]}
data_score = pd.DataFrame(data_score)

plt.figure(figsize=(12,8))
plt.style.use("ggplot")
plt.bar(x='names',height='scores',data=data_score, color="#dc7633")
plt.yticks(np.arange(0,110,10), rotation=45)
plt.title("MODEL SCORES")
plt.grid(True)
plt.show()

<a id="feature_engineering"></a>
# Feature Engineering

for accuracy or discrimination the most important part is feature engineering. We have to use math and statistical skill in there.

In [ ]:
data = data.rename(columns = {'PAY_0': 'PAY_1'}, inplace = False)

# Payment delays.
1. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.
default payment (Yes = 1, No = 0)

#-2: Tüketim yok; -1: Tam olarak ödendi; 0: Döner kredi kullanımı; 1 = bir aylık ödeme gecikmesi; 2 = iki aylık ödeme gecikmesi; . . .; 8 = sekiz aylık ödeme gecikmesi; 9 = dokuz ay ve üzeri ödeme gecikmesi.

In [ ]:
payment_data=['PAY_1','PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
data[payment_data]
print(data["PAY_1"].value_counts())
print( len(data[(data["PAY_1"]>=1) & (data["target"]==1)]) )   #nice for new feature
print( len(data[(data["PAY_1"]>=1) & (data["target"]==0)]) )

3. 0 and 2 values is significant for understanding payment flow. But 0 is not important for us because of that every people can payment regular for a month in 6 month.

In [ ]:
def investing_pay_1():
    for j in range(1,6):
        print("Payment_"+str(j))
        for i in range(-2,8,1):
            pay_contains = len(data[(data["PAY_"+str(j)]==i) & (data["target"]==1)])
            print(str(i)+"_quantity: ", pay_contains)
            
investing_pay_1()
data["payment_1_eng"]=[1 if i<0 else 0 if i==0 else 2 for i in data["PAY_1"]]

# Aykırı değerleri boxplotla görselleştirebiliriz aynı zamanda çeşitli formülasyonlar ile bu outlierları tespit edebiliriz.

In [ ]:
sns.distplot(data['AGE'])
print("Skewness(çarpıklık): %{:.3f}".format(data['AGE'].skew()))
print("Kurtosis(basıklık): %{:.3f}".format(data['AGE'].kurt()))

In [ ]:
plt.style.use("ggplot")
fig, axs = plt.subplots(2,3, figsize=(16,10))

axs[0,0].boxplot(data["AGE"])
axs[0,0].set_title("Age outliers", color="#14B0B3", size=20)
axs[0,0].set_ylabel("Age", color="#36EA15", size = 15)

axs[0,1].boxplot(data["LIMIT_BAL"])
axs[0,1].set_title("Limit Amount", color="#14B0B3", size=20)
axs[0,1].set_ylabel("credit_limit", color="#258912", size = 15)

axs[0,2].boxplot(data["PAY_1"])
axs[0,2].set_title("PAY 1", color="#14B0B3", size=20)
axs[0,2].set_ylabel("Pay_1 Amount", color="#16560A", size = 15)

axs[1,0].boxplot(data["PAY_AMT1"])
axs[1,0].set_title("PAY Amount 1", color="#14B0B3", size=20)
axs[1,0].set_ylabel("Pay_1 Amount", color="#36EA15", size = 15)

axs[1,1].boxplot(data["PAY_AMT2"])
axs[1,1].set_title("PAY Amount 2", color="#14B0B3", size=20)
axs[1,1].set_ylabel("Pay_1 Amount", color="#258912", size = 15)

axs[1,2].boxplot(data["BILL_AMT1"])
axs[1,2].set_title("Bill Amount 1", color="#14B0B3", size=20)
axs[1,2].set_ylabel("Pay_1 Amount", color="#16560A", size = 15)


1. ımmm its looks like normal distribution but ı think some outliers on the right side.
2. at the same time the graph skewed distribution to the right

# outlier önce tespit edip ardından bu değerlerin target yüzdelerine bakalım

# Now lets catch outliers!

In [ ]:
outlier_columns=["AGE","LIMIT_BAL", "PAY_1", "PAY_2","PAY_3","PAY_4","PAY_5","PAY_6", "PAY_AMT1", "PAY_AMT2","BILL_AMT1","BILL_AMT2"]
age_outlier = []
limitbal_outlier = []
pay1_outlier = []
pay2_outlier = []
pay3_outlier = []
pay4_outlier = []
pay5_outlier = []
pay6_outlier = []
payamt1_outlier = []
payamt2_outlier = []
billamt_outlier = []
billamt2_outlier = []

outliers_array=[age_outlier,limitbal_outlier,pay1_outlier,pay2_outlier,pay3_outlier, pay4_outlier, pay5_outlier, pay6_outlier,payamt1_outlier,payamt2_outlier,billamt_outlier,billamt2_outlier]

def outlier_detect(data):
    for i, column in enumerate(outlier_columns):
        Q1 = np.percentile(data[column], 25)
        Q3 = np.percentile(data[column], 75)
        IQR = Q3- Q1
        outlier_step = IQR*1.5

        outlier_values = data[(data[column]<Q1-outlier_step) | (data[column] >Q3+outlier_step)]
        print("*****", column, "******")
        #print(outlier_values)
        
        outlier_values_and_1 = data[((data[column]<Q1-outlier_step) | (data[column] >Q3+outlier_step)) & (data["target"]==1)]
        print("1 ratio is: %",((len(outlier_values_and_1)/len(outlier_values))*100))
        
        outliers_array[i].append(outlier_values_and_1.index)

outlier_detect = outlier_detect(data)

<a id="feature_extraction"></a>
# feature extraction

1. about payment one

In [ ]:
data["dangeroues_cust"]=0

for i in pay1_outlier:
    data["dangeroues_cust"].iloc[i]=1

print(data["dangeroues_cust"].value_counts())

2. about education

In [ ]:
data["education_level"]=[0 if i == 0 or i==4 or i==5 else 1 if i ==6 or i==1 else 2  for i in data["EDUCATION"]]

3.about limit bal.

In [ ]:
limit_mean = data["LIMIT_BAL"].mean()
limit_std = data["LIMIT_BAL"].std()
print("Credit limit mean: {:.2f} and standart deviation: {:.2f}".format(limit_mean, limit_std))

Q1= data["LIMIT_BAL"].quantile(.20) 
Q2= data["LIMIT_BAL"].quantile(.40) 
Q3= data["LIMIT_BAL"].quantile(.60) 
Q4= data["LIMIT_BAL"].quantile(.80) 
data["limit_class"] = [0 if i<Q1 else 1 if i>=Q1 and i<Q2 else 2 if i>=Q2 and i<Q3 else 4 if i>=Q3 and i<Q4 else 5 for i in data["LIMIT_BAL"]]

4.pay amount

In [ ]:
data["dangeroues_cust2"]=0

for i in payamt2_outlier:
    data["dangeroues_cust2"].iloc[i]=1

print(data["dangeroues_cust2"].value_counts())

5. repay_cust

In [ ]:
data["total_payment"]=data["PAY_AMT1"]+data["PAY_AMT2"]+data["PAY_AMT3"]+data["PAY_AMT4"]+data["PAY_AMT5"]+data["PAY_AMT6"]

data["repay_cust"]=1
repay_cust = data[(data["total_payment"]>data["LIMIT_BAL"]) & (data["target"]==0)].index
for i in repay_cust:
    data["repay_cust"].iloc[i]=0

6. pay grades means

In [ ]:
data["pays_means"]=0
def mean_grades(data, column):
    for i in range(1,7):
        data[column] += data["PAY_"+str(i)]
    data[column]=data[column]/i

mean_grades(data, "pays_means")


pay_mean = data["pays_means"].mean()
pay_std = data["pays_means"].std()
pay_min = data["pays_means"].min()
pay_max = data["pays_means"].max()
print("for grades:\n pay mean:{:.2f}\n pay standart dev.:{:.2f}\n pay min: {}\n pay max: {}".format(pay_mean,pay_std,pay_min, pay_max))
data["pays_means"]=[0 if i<(pay_mean-pay_std) else 2 if i>(pay_mean+pay_std) else 1 for i in data["pays_means"] ]

7. Age Classes 

In [ ]:
Q1= data["AGE"].quantile(.20) 
Q2= data["AGE"].quantile(.40) 
Q3= data["AGE"].quantile(.60) 
Q4= data["AGE"].quantile(.80) 
data["AGE_class"] = [0 if i<Q1 else 1 if i>=Q1 and i<Q2 else 2 if i>=Q2 and i<Q3 else 4 if i>=Q3 and i<Q4 else 5 for i in data["AGE"]]

<a id="different_data"></a>
# Creating Different Dataset

In [ ]:
data.head()

In [ ]:
data2=pd.DataFrame(data.iloc[:,23:])
plt.figure(figsize=(18,10))
sns.heatmap(data2.corr(),annot=True,fmt='.2f',color='red',cmap='coolwarm')
plt.show()

In [ ]:
data2 = pd.concat([data[["MARRIAGE","PAY_1"]], data2],axis=1)
data2

# Dont forget feature scaling for logistic regression !!!

In [ ]:
X_2=data2.drop(columns=['target'],axis=1)
y_2=data2.iloc[:,2]

#scaled = (x - xmin) / (xmax - xmin)  for logistic regression
X_2["total_payment"] = (X_2["total_payment"]- X_2["total_payment"].min()) /  (X_2["total_payment"].max()- X_2["total_payment"].min()) 
X_train_2,X_test_2,y_train_2,y_test_2=train_test_split(X_2,y_2,test_size=0.35,random_state=0)

In [ ]:
xgb.fit(X_train_2, y_train_2)
y_pred_xgb =xgb.predict(X_test_2)
score_xgb_2=accuracy_score(y_pred_xgb,y_test_2)*100
print("Xgboost SCORE:{:.3f}".format(score_xgb_2))

rfc.fit(X_train_2, y_train_2)
y_pred_rfc = rfc.predict(X_test_2)
score_rfc_2=accuracy_score(y_pred_rfc,y_test_2)*100
print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc_2))

log_reg = LogisticRegression(n_jobs=-1)
log_reg.fit(X_train_2, y_train_2)
y_pred_log_2 = log_reg.predict(X_test_2)
score_log_2=accuracy_score(y_pred_log_2,y_test_2)*100
print("Logistic Regression SCORE:{:.3f}".format(score_log_2))

In [ ]:
X_3=data.drop(columns=['AGE','target','PAY_6', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6','BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6'],axis=1)
y=data.iloc[:,23]


data3 = pd.concat([X_3, y], axis=1)
X_train_3,X_test_3,y_train_3,y_test_3=train_test_split(X_3,y,test_size=0.35,random_state=0)

In [ ]:
data3

# data3 Models

In [ ]:
xgb.fit(X_train_3, y_train_3)
y_pred_xgb_ =xgb.predict(X_test_3)
score_xgb_3=accuracy_score(y_pred_xgb_,y_test_3)*100
print("XgboostClassifier SCORE:{:.3f}".format(score_xgb_3))
print("Xgb Confusion matrix")
cm_xgb = confusion_matrix(y_test_3, y_pred_xgb_)
print(cm_xgb)

rfc.fit(X_train_3, y_train_3)
y_pred_rfc = rfc.predict(X_test_3)
score_rfc_3=accuracy_score(y_pred_rfc,y_test_3)*100
print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc_3))
print("RandomForestClassifier matrix")
cm_rfc = confusion_matrix(y_test_3, y_pred_rfc)
print(cm_rfc)

<a id="Dim_reduction"></a>
# Dimension Deduction

# for the more information about this part you should go to [StatQuest](https://www.youtube.com/user/joshstarmer) youtube channel!

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.patches as mpatches
import time

In [ ]:
x_original = data.loc[data["target"]==0].drop(["target"], axis=1)
x_original2 = data.loc[data["target"]==1].drop(["target"], axis=1)

x_data = data2.loc[data2["target"]==0].drop(["target"], axis=1)
x_data2 = data2.loc[data2["target"]==1].drop(["target"], axis=1)

x_data3 = data3.loc[data3["target"]==0].drop(["target"], axis=1)
x_data_3 = data3.loc[data3["target"]==1].drop(["target"], axis=1)

In [ ]:
#PCA implementation for original data
t1=time.time()
pca_reduc_X1 = PCA(n_components=2, random_state=0).fit_transform(x_original.values)
pca_reduc_y1 = PCA(n_components=2, random_state=0).fit_transform(x_original2.values)
t2=time.time()
print("PCA time: {:.3f}".format(t2-t1))

In [ ]:
#PCA implementation for data2
t1=time.time()
pca_reduc_X = PCA(n_components=2, random_state=0).fit_transform(x_data.values)
pca_reduc_y = PCA(n_components=2, random_state=0).fit_transform(x_data2.values)
t2=time.time()
print("PCA time: {:.3f}".format(t2-t1))

In [ ]:
#PCA implementation for data3
t1=time.time()
pca_reduc_X3 = PCA(n_components=2, random_state=0).fit_transform(x_data3.values)
pca_reduc_y3 = PCA(n_components=2, random_state=0).fit_transform(x_data_3.values)
t2=time.time()
print("PCA time: {:.3f}".format(t2-t1))

In [ ]:
#TSNE implementation for data 3
t1=time.time()
tsne_reduc_X = TSNE(n_components=2, random_state=2).fit_transform(x_data3.values)
tsne_reduc_y = TSNE(n_components=2, random_state=2).fit_transform(x_data_3.values)
t2=time.time()
print("T-SNE time: {:.3f}".format(t2-t1))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(28,16))
fig.suptitle("Dimension Reduction with PCA and T-SNE", size=20)

axs[0,0].scatter(pca_reduc_X1[:,0], pca_reduc_X1[:,1], color="#FAAC58", label=("target=0"), linewidth=0.5, alpha=0.7, edgecolors="black")
axs[0,0].scatter(pca_reduc_y1[:,0], pca_reduc_y1[:,1], color="#210B61", label=("target=1"), linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[0,0].set_title("Original Data Scatter with *PCA*", color="#FF0040", size=20)
axs[0,0].legend()

axs[0,1].scatter(pca_reduc_X[:,0], pca_reduc_X[:,1], color="#FAAC58", label=("target=0"), linewidth=0.5, alpha=0.7, edgecolors="black")
axs[0,1].scatter(pca_reduc_y[:,0], pca_reduc_y[:,1], color="#210B61", label=("target=1"), linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[0,1].set_title("Created Data2 Scatter with *PCA*", color="#FF0040", size=20)
axs[0,1].legend()

axs[1,0].scatter(pca_reduc_X3[:,0], pca_reduc_X3[:,1], color="#FAAC58", label=("target=0"), linewidth=0.5, alpha=0.7, edgecolors="black")
axs[1,0].scatter(pca_reduc_y3[:,0], pca_reduc_y3[:,1], color="#210B61", label=("target=1"), linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[1,0].set_title("Created Data3 Scatter with *PCA*", color="#FF0040", size=20)
axs[1,0].legend()

axs[1,1].scatter(tsne_reduc_X[:,0], tsne_reduc_X[:,1], color="#F4E80E", label=("target=0"), linewidth=0.5, alpha=0.7, edgecolors="black")
axs[1,1].scatter(tsne_reduc_y[:,0], tsne_reduc_y[:,1], color="#F1140D", label=("target=1"), linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[1,1].set_title("Created Data3 Scatter with *T-SNE*", color="#FF0040", size=20)
axs[1,1].legend()

# I make different data with different features. After that I observe their distributions. I determine data3(with PCA) is more processable. 

# Does this way make sense?


<a id="under_samp"></a>
# Undersampling process

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NearMiss

In [ ]:
enn= EditedNearestNeighbours(sampling_strategy="majority", n_neighbors=2)
TomekLinks = TomekLinks(sampling_strategy="not minority")
NearMiss = NearMiss(version=2)



def under_sampling_function(X_train_3, y_train_3, model, removed_array):

    x_resampling, y_resampling = model.fit_resample(X_train_3, y_train_3)
    removed_array = model.sample_indices_
    print("*"*25)
    print("Models Resampling score with", str(model))

    xgb = XGBClassifier()
    xgb.fit(x_resampling, y_resampling)
    y_pred_xgb =xgb.predict(X_test_3)
    score_xgb=accuracy_score(y_pred_xgb,y_test_3)*100
    print("XGBoost Classifier SCORE:{:.3f}".format(score_xgb))
    cm_xgb = confusion_matrix(y_test_3, y_pred_xgb)
    print(cm_xgb)
    

    rfc = RandomForestClassifier()
    rfc.fit(x_resampling, y_resampling)
    y_pred_rfc = rfc.predict(X_test_3)
    score_rfc=accuracy_score(y_pred_rfc,y_test_3)*100
    print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc))
    cm_rfc = confusion_matrix(y_test_3, y_pred_rfc)
    print(cm_rfc,"\n")
    

    
    return removed_array, x_resampling, y_resampling, y_pred_xgb, y_pred_rfc


removed_data_enn = []
removed_data_tmk = []
removed_data_NearM = []

#bir liste oluşturalım ve fonksiyon parametresi olarak ekleyelim
edited_nearest_neighbours = under_sampling_function(X_train_3, y_train_3, enn, removed_data_enn)
tomeklinks_undersampling = under_sampling_function(X_train_3, y_train_3, TomekLinks, removed_data_tmk)
nearmiss_undersampling = under_sampling_function(X_train_3, y_train_3, NearMiss, removed_data_NearM)

In [ ]:
edited_nearest_neighbours_x = pd.DataFrame(edited_nearest_neighbours[1])
edited_nearest_neighbours_y = pd.DataFrame(edited_nearest_neighbours[2])

tomeklinks_undersampling_x = pd.DataFrame(tomeklinks_undersampling[1])
tomeklinks_undersampling_y = pd.DataFrame(tomeklinks_undersampling[2])

nearmiss_undersampling_x = pd.DataFrame(nearmiss_undersampling[1])
nearmiss_undersampling_y = pd.DataFrame(nearmiss_undersampling[2])

In [ ]:
data_enn = pd.concat([edited_nearest_neighbours_x,edited_nearest_neighbours_y], axis=1)
data_tomek = pd.concat([tomeklinks_undersampling_x,tomeklinks_undersampling_y], axis=1)
data_nearmiss = pd.concat([nearmiss_undersampling_x,nearmiss_undersampling_y], axis=1)

In [ ]:
#removed data for different under_sampling tecknique
print("number of removed data with EditedNearestNeighbours :{}".format(data_enn.shape[0]))
print("number of removed data with TomekLinks :{}".format(data_tomek.shape[0]))
print("number of removed data with Nearmiss(version2) :{}".format(data_nearmiss.shape[0]))

<a id="techniques"></a>

# 7. UnderSampling visualization

In [ ]:
x_0 = data3.loc[data["target"]==0].drop(["target"], axis=1)
x_1 = data3.loc[data["target"]==1].drop(["target"], axis=1)
pca_reduc_X = PCA(n_components=2, random_state=0).fit_transform(x_0.values)
pca_reduc_y = PCA(n_components=2, random_state=0).fit_transform(x_1.values)

x_0 = data_enn.loc[data_enn["target"]==0].drop(["target"], axis=1)
x_1 = data_enn.loc[data_enn["target"]==1].drop(["target"], axis=1)
pca_reduc_X_enn =PCA(n_components=2, random_state=0).fit_transform(x_0.values)
pca_reduc_y_enn = PCA(n_components=2, random_state=0).fit_transform(x_1.values)

x_0 = data_tomek.loc[data_tomek["target"]==0].drop(["target"], axis=1)
x_1 = data_tomek.loc[data_tomek["target"]==1].drop(["target"], axis=1)
pca_reduc_X_tomek = PCA(n_components=2, random_state=0).fit_transform(x_0.values)
pca_reduc_y_tomek = PCA(n_components=2, random_state=0).fit_transform(x_1.values)

x_0 = data_nearmiss.loc[data_nearmiss["target"]==0].drop(["target"], axis=1)
x_1 = data_nearmiss.loc[data_nearmiss["target"]==1].drop(["target"], axis=1)
pca_reduc_X_nearmiss = PCA(n_components=2, random_state=0).fit_transform(x_0.values)
pca_reduc_y_nearmiss = PCA(n_components=2, random_state=0).fit_transform(x_1.values)



fig, axs = plt.subplots(2, 2, figsize=(24,18))

axs[0,0].scatter(pca_reduc_X[:,0], pca_reduc_X[:,1], color="#FAAC58", label=0, linewidth=0.5, alpha=0.7, edgecolors="black")
axs[0,0].scatter(pca_reduc_y[:,0], pca_reduc_y[:,1], color="#210B61", label=1, linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[0,0].set_title("Original Data Distribution", color="#FF0040", size=20)
axs[0,0].set_xlabel("number of data: {}".format(len(pca_reduc_X) +len(pca_reduc_y)))
axs[0,0].spines['top'].set_visible(False)
axs[0,0].spines['right'].set_visible(False)
axs[0,0].get_xaxis().tick_bottom()
axs[0,0].get_yaxis().tick_left()


axs[0,1].scatter(pca_reduc_X_enn[:,0], pca_reduc_X_enn[:,1], color="#FAAC58", label=0, linewidth=0.5, alpha=0.7, edgecolors="black")
axs[0,1].scatter(pca_reduc_y_enn[:,0], pca_reduc_y_enn[:,1], color="#210B61", label=1, linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[0,1].set_title("Edited Nearest Neighbours", color="#FF0040", size=20)
axs[0,1].set_xlabel("number of data: {}".format(len(pca_reduc_X_enn)+len(pca_reduc_y_enn)))
    
axs[1,0].scatter(pca_reduc_X_tomek[:,0], pca_reduc_X_tomek[:,1], color="#FAAC58", label=0, linewidth=0.5, alpha=0.7, edgecolors="black")
axs[1,0].scatter(pca_reduc_y_tomek[:,0], pca_reduc_y_tomek[:,1], color="#210B61", label=1, linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[1,0].set_title("TomekLinks", color="#FF0040", size=20)
axs[1,0].set_xlabel("number of data: {}".format(len(pca_reduc_X_tomek)+len(pca_reduc_y_tomek)))

axs[1,1].scatter(pca_reduc_X_nearmiss[:,0], pca_reduc_X_nearmiss[:,1], color="#FAAC58", label=0, linewidth=0.5, alpha=0.7, edgecolors="black")
axs[1,1].scatter(pca_reduc_y_nearmiss[:,0], pca_reduc_y_nearmiss[:,1], color="#210B61", label=1, linewidth=1, alpha=0.8, edgecolors="#E0F8E0")
axs[1,1].set_title("Nearmiss", color="#FF0040", size=20)
axs[1,1].set_xlabel("number of data: {}".format(len(pca_reduc_X_nearmiss)+len(pca_reduc_y_nearmiss)))

<a id="GridSearch"></a>
# GridSearch 

In [ ]:
from sklearn.model_selection import GridSearchCV 

In [ ]:
#randomforest grid_search
def rfc_gridsearch(Model, X_Train, y_train):
    
    param_grid_rfc = {"n_estimators":[100, 120, 140],
                     "criterion": ["gini","entropy"],
                     "min_samples_leaf":list(range(3,5,1))}

    grid_search = GridSearchCV(estimator=Model,
                              param_grid = param_grid_rfc,
                              scoring="accuracy")

    grid_search.fit(X_Train,y_train) 
    rfc_best_score=grid_search.best_score_
    best_parameters=grid_search.best_params_
                      
    return Model, best_parameters

rfc_gridSearch_def = rfc_gridsearch(rfc, edited_nearest_neighbours[1], edited_nearest_neighbours[2])
print("Random Forest Best Parameters: {}".format(rfc_gridSearch_def[1]))

In [ ]:
rfc = RandomForestClassifier(criterion='entropy', min_samples_leaf=4, n_estimators=140)
rfc.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])
y_pred_rfc = rfc.predict(X_test_3)

score_rfc_enn_grid=accuracy_score(y_pred_rfc,y_test_3)*100
print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc_enn_grid))

cm_rfc = confusion_matrix(y_test_3, y_pred_rfc)
print(cm_rfc)

In [ ]:
#xgb grid search
def xgb_gridSearch(Model, X_train, y_train):
    param_grid_xgb = { "gamma": [0.1, 0.5, 1, 2],
                       "learning_rate": [0.1, 1, 5],
                       "max_depth": [1, 5, 10]}

    grid_search = GridSearchCV(estimator=Model,
                              param_grid = param_grid_xgb,
                              scoring="accuracy")

    grid_search.fit(X_train,y_train) 

    xgb_best_score=grid_search.best_score_
    best_parameters=grid_search.best_params_
    
    return Model, best_parameters

xgb_gridSearch_def_1 = xgb_gridSearch(xgb,  edited_nearest_neighbours[1], edited_nearest_neighbours[2])
print("XGBoost Best Parameters for ENN: {}".format(xgb_gridSearch_def_1[1]))

In [ ]:
xgb = XGBClassifier(gamma= 0.1 ,learning_rate=0.1, max_depth=5)
xgb.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])
y_pred_xgb =xgb.predict(X_test_3)

score_xgb_enn_grid=accuracy_score(y_pred_xgb,y_test_3)*100
print("XGboost SCORE:{:.3f}".format(score_xgb_enn_grid))

cm_xgb = confusion_matrix(y_test_3, y_pred_xgb)
print(cm_xgb)

# xgb gridsearch did not change accuracy score effectively but it made precision and recall values more useful for our aim.

<a id="F1_scores"></a>
# lets look at the F1 scores

In [ ]:
from sklearn.model_selection import cross_val_score

model_list=[rfc, xgb]

all_cross_scores=[]


for i in model_list:
    scores=cross_val_score(i, edited_nearest_neighbours[1], edited_nearest_neighbours[2], cv=5, n_jobs=-1)
    print(str(i),", scores: ", scores)
    all_cross_scores.append(scores)  
    print("Mean cross validation score of {} models : {:.2f}".format(i, scores.mean() ))
    print("*"* 25)
    

    
xgb_cross_score = all_cross_scores[0]
rfc_cross_score = all_cross_scores[1]


plt.style.use("ggplot")
fig, axs = plt.subplots(1, 2, figsize=(24,8))


axs[0].plot(rfc_cross_score, linewidth=3, color="black", marker='o')
axs[0].hlines(rfc_cross_score.mean(),xmin=0, xmax=4, color="red", alpha=0.4, linewidth=2)
axs[0].set_xlabel("iterations",  color="red", size=24)
axs[0].set_title("RandomForest", color="#FF69B4")
axs[0].grid(True)

axs[1].plot(xgb_cross_score, linewidth=3, color="black", marker='o')
axs[1].set_title("XGBoost", color="#FF69B4")
axs[1].hlines(xgb_cross_score.mean(),xmin=0, xmax=4, color="red", alpha=0.4, linewidth=2)
axs[1].grid(True)

<a id="over_sampling"></a>
# Over Sampling Process

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import KMeansSMOTE
from imblearn.pipeline import Pipeline

In [ ]:
smote = SMOTE(sampling_strategy=0.3)
adsyn = ADASYN(n_neighbors=3)



def over_sampling_function(X_train, y_train, model):
    
    x_resampling, y_resampling = model.fit_resample(X_train_3, y_train_3)
    
    print("*"*25)
    print("Models Resampling score with", str(model))

    xgb = XGBClassifier()
    xgb.fit(x_resampling, y_resampling)
    y_pred_xgb_over =xgb.predict(X_test_3)
    score_xgb=accuracy_score(y_pred_xgb_over,y_test_3)*100
    print("XGBoost Classifier SCORE:{:.3f}".format(score_xgb))
    cm_xgb = confusion_matrix(y_test_3, y_pred_xgb_over)
    print(cm_xgb)

    rfc = RandomForestClassifier()
    rfc.fit(x_resampling, y_resampling)
    y_pred_rfc_over = rfc.predict(X_test_3)
    score_rfc=accuracy_score(y_pred_rfc_over,y_test_3)*100
    print("RandomForestClassifier SCORE:{:.3f}".format(score_rfc))
    cm_rfc = confusion_matrix(y_test_3, y_pred_rfc_over)
    print(cm_rfc,"\n")
    
    return y_pred_xgb_over, y_pred_rfc_over, x_resampling,y_resampling



smote_function=over_sampling_function(X_train_3, y_train_3, smote) 
adasyn_function=over_sampling_function(X_train_3, y_train_3, adsyn)

<a id="over_sampling_f1"></a>
# F1 scores  of Over Sampling

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
f1_score_xgb_2  = classification_report(y_test_3, smote_function[0], digits=3)
print("*"*20,"XGBoost scores with SMOTE","*"*20,"\n",f1_score_xgb_2)

f1_score_xgb  = classification_report(y_test_3, adasyn_function[0], digits=3)
print("*"*20,"XGBoost scores with SMOTE","*"*20,"\n",f1_score_xgb)

# we have some datasets and teckniques. recall precions we başarı yüzdelerini dikkate aldığımızda bu modelleri ve şu teknikleri seçtik.

In [ ]:
xgb = XGBClassifier(gamma= 0.1, learning_rate=0.1, max_depth=5)
xgb.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])
xgb_pred_enn = xgb.predict(X_test_3)
score_xgb_enn=accuracy_score(xgb_pred_enn,y_test_3)*100


rfc = RandomForestClassifier(criterion='entropy', min_samples_leaf=4, n_estimators=140)
rfc.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])
rfc_pred_enn = rfc.predict(X_test_3)
score_rfc_enn=accuracy_score(rfc_pred_enn,y_test_3)*100


xgb_2 = XGBClassifier() 
xgb_2.fit(nearmiss_undersampling[1], nearmiss_undersampling[2])  



fig, axs = plt.subplots(1, 3, figsize=(28,10))

plot_confusion_matrix(xgb, X_test_3, y_test_3 ,cmap=plt.cm.copper, ax=axs[0],display_labels=("not default", "default"))
axs[0].set_title("XGBoost/ EditedNearestNeighbours",fontsize=22, color="#FF0000")
axs[0].grid(False)


plot_confusion_matrix(rfc, X_test_3, y_test_3 ,cmap=plt.cm.hot, ax=axs[1],display_labels=("not default", "default"))
axs[1].set_title("RandomFC/ EditedNearestNeighbours",fontsize=22, color="#FF0000")
axs[1].grid(False)

plot_confusion_matrix(xgb_2, X_test_3, y_test_3 ,cmap=plt.cm.magma, ax=axs[2],display_labels=("not default", "default"))
axs[2].set_title("XGBoost Classifier / NearMiss",fontsize=22, color="#FF0000")
axs[2].grid(False)

# Top 3 classification_report

In [ ]:
f1_score_xgb_enn  = classification_report(y_test_3, y_pred_xgb, digits=3)
print("*"*20,"XGB scores with EditedNearestNeighbours","*"*20,"\n",f1_score_xgb_enn)

f1_score_rfc_enn  = classification_report(y_test_3, y_pred_rfc, digits=3)
print("*"*20,"RandomForest scores with EditedNearestNeighbours","*"*20,"\n",f1_score_rfc_enn)

f1_score_rfc_nearM  = classification_report(y_test_3, nearmiss_undersampling[3], digits=3)
print("*"*20,"RandomForest scores with NearMiss","*"*20,"\n",f1_score_rfc_nearM)

<a id="roc_curve"></a>
# Roc Curve

xgb = XGBClassifier(gamma= 0.1, learning_rate=0.1, max_depth=10)

xgb.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])

rfc = RandomForestClassifier(criterion='gini', min_samples_leaf=3, n_estimators=120)

rfc.fit(edited_nearest_neighbours[1], edited_nearest_neighbours[2])

xgb_2 = XGBClassifier(gamma= 1, learning_rate=0.1, max_depth=5) 

xgb_2.fit(nearmiss_undersampling[1], nearmiss_undersampling[2])  

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

In [ ]:
probs = xgb.predict_proba(X_test_3)
preds = probs[:,1]
xgb_fpr, xgb_tpr, lof_threshold_xgb = roc_curve(y_test_3, preds)
roc_auc_xgb = metrics.auc(xgb_fpr, xgb_tpr)

probs = rfc.predict_proba(X_test_3)
preds = probs[:,1]
rfc_fpr, rfc_tpr, lof_threshold_rfc = roc_curve(y_test_3, preds)
roc_auc_rfc=roc_auc_score(y_test_3, preds)

probs = xgb_2.predict_proba(X_test_3)
preds = probs[:,1]
xgb_fpr_2, xgb_tpr_2, lof_threshold_xgb = roc_curve(y_test_3, preds)
roc_auc_xgb_2 = metrics.auc(xgb_fpr_2, xgb_tpr_2)


def graph_roc_curve_multiple(xgb_fpr, xgb_tpr,rfc_fpr, rfc_tpr):
    
    plt.figure(figsize=(16,12))
    plt.title("ROC Curve \n TOP 3 Classifiers using X_Train3", fontsize=18)
    
    plt.plot(xgb_fpr, xgb_tpr, color="red",    marker=".", markersize=3, markeredgecolor ="white", markeredgewidth=.1, label="AUC XGboost C. with ENN:{:.4f}".format(roc_auc_xgb))
    plt.plot(rfc_fpr, rfc_tpr, color="purple", marker=".", markersize=3, markeredgecolor ="white", markeredgewidth=.1, label="AUC RandomFC with ENN:{:.4f}".format(roc_auc_rfc))
    plt.plot(xgb_fpr_2, xgb_tpr_2, color="yellow", marker="o", markersize=3, markeredgecolor ="green", markeredgewidth=.1, label="AUC XGboost C. with Nearmiss:{:.4f}".format(roc_auc_xgb_2))
    
    plt.plot([0,1], [0,1], "k--")
    plt.axis([-0.01, 1, 0, 1.01])
    #eksenler ile çizim konumlarını belirler
    plt.xlabel("False Positive Rate", fontsize=16)
    plt.ylabel("True Positive Rate", fontsize=16)

    plt.legend()
    plt.show()
    
graph_roc_curve_multiple(xgb_fpr, xgb_tpr,rfc_fpr, rfc_tpr)


<a id="Conclusions"></a>
# Datasets about Conclusions

In [ ]:
scores = {'X_train':   [round(score_rfc,3), round(score_xgb,3)],
          'X_train_2': [round(score_rfc_2,3), round(score_xgb_2,3)],
          'X_train_3':  [round(score_rfc_3,3), round(score_xgb_3,3)],
          "Scores with ENN (over_sampling)":[round(score_rfc_enn,3), round(score_xgb_enn,3)],
           "Scores with ENN and GridSearchng)":[round(score_rfc_enn_grid,3), round(score_xgb_enn_grid,3)]}

pd.DataFrame(index=['Random Forest Classifier', 'XGboost Classifier'],
             data = scores)

# so the dataset is the shows that why we use X_train_3 and ENN. little Bam ?
# but the answer is not just about Accuracy scores lets look over precision Scores

1. The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives.
2. The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
xgb_precision_score  = precision_score(y_pred_xgb_, y_test_3)
xgb_enn_precision  = precision_score(xgb_pred_enn, y_test_3)
print("Xgboost Classifier precision score without enn technique :{:.3f} ".format(xgb_precision_score))
print("Xgboost Classifier precision score with enn technique :{:.3f} ".format(xgb_enn_precision))

print("*"*50)
rfc_precision_score  = precision_score(y_pred_rfc, y_test_3)
rfc_enn_precision  = precision_score(rfc_pred_enn, y_test_3)
print("Random Forest Classifier precision score without enn technique :{:.5f} ".format(rfc_precision_score))
print("Random Forest Classifier  precision score with enn technique :{:.5f} ".format(rfc_enn_precision))

# X_train_3 Precision Scores.  (I Think this dataset is Double Bam!!!) Thanks the  [statquest with Josh Starmer](https://www.youtube.com/user/joshstarmer) for Great informations.

In [ ]:
scores = {"Precision Score of without ENN":[round(rfc_precision_score,4), round(xgb_precision_score,4)],
         "Precision Score with ENN":[round(rfc_enn_precision,4), round(xgb_enn_precision,4)]}

pd.DataFrame(index=['Random Forest Classifier', 'XGboost Classifier'],
             data = scores)